In [7]:
import pandas as pd
import re
from nltk.corpus import stopwords
from gensim import similarities

import math
import time
import datetime
import numpy as np

from scipy import spatial
import ast

from tenacity import (
    retry,
    stop_after_attempt,
    wait_random_exponential,
)  # for exponential backoff

from sklearn.metrics import classification_report
from sklearn.metrics import PrecisionRecallDisplay

import gensim
import nltk
from nltk.tokenize import sent_tokenize
import string
from nltk.corpus import stopwords
from gensim.models import Word2Vec
from openai.embeddings_utils import cosine_similarity

from gensim.models import Word2Vec
from gensim import similarities
from sklearn.metrics.pairwise import cosine_similarity
from openai.embeddings_utils import cosine_similarity, get_embedding
import tensorflow as tf
import tensorflow_hub as hub 

## Embed sentence 

In [3]:
df = pd.read_excel('amazon_review JOIN.xlsx')

In [10]:
df['Review Content new'] = df['Review Content'].apply(lambda x: str(x) if x is not None else '') 

In [11]:
def clean_text(text):
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    text = text.lower()
    return text

df['Cleaned Reviews'] = df['Review Content new'].apply(clean_text)

In [12]:
def remove_stopwords(text):
    stop_words = set(stopwords.words('english'))
    words = text.split()
    filtered_words = [word for word in words if word.lower() not in stop_words]
    return ' '.join(filtered_words)

In [13]:
df['Final Reviews'] = df['Cleaned Reviews'].apply(remove_stopwords)

In [14]:
# def split_text(text):
#     sentences = sent_tokenize(str(text))
#     return sentences 

# def remove_punc(text):
#     remove = str.maketrans('','',string.punctuation)
#     remove_punc = [sentence.translate(remove) for sentence in text]
#     return remove_punc

def brand(model):
    match = re.search(r'^(HP|Epson|Canon)', model, re.IGNORECASE)
    if match:
        return match.group(0)

In [15]:
df['Brand'] = df['Review Model'].apply(brand)
# df['Review Content split'] =  df['Review Content'].apply(split_text)

In [16]:
df.head(2)

,Review Model,Retailer,Comp Model,Review date,Review name,Review rating,Review title,Review Content,Verified Purchase or not,People_find_helpful,vine or not,URL,list price,rating count,overall rating,Brand,Review Content new,Cleaned Reviews,Final Reviews
0,Canon Pixma TS6420a,Amazon,NaN,2023-08-10,Ernest Birkholz,5,Works great,🖨 was easy to install and works great.,Verified Purchase,NaN,NaN,https://www.amazon.com/product-reviews/B09TG52...,129.99,285,4.2,Canon,🖨 was easy to install and works great.,was easy to install and works great,easy install works great
1,HP OfficeJet Pro 9015e,Amazon,Epson - Workforce 4820,2022-06-04,mattey,3,spunky mid size printer,🔆Slower print speed than what I’m used to (old...,Verified Purchase,NaN,NaN,https://www.amazon.com/product-reviews/B08QR6P...,289.99,285,4.2,HP,🔆Slower print speed than what I’m used to (old...,slower print speed than what im used to old mo...,slower print speed im used old modelhpojpro ne...


In [17]:
model = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")

def embed(text,model):
    embeddings = model(text)
    return [embedding.numpy() for embedding in embeddings]


df['Embed_sentence'] = embed(df['Final Reviews'], model)

In [18]:
df['Embed_sentence']

0        [-0.047439758, -0.03413595, -6.694528e-05, 0.0...
1        [-0.07037805, -0.06696488, -0.07535003, 0.0402...
2        [-0.035656825, -0.033195715, 0.07047429, 0.073...
3        [-0.04945306, -0.064851016, -0.0635704, 0.0535...
4        [-0.016937636, -0.046409328, -0.05469985, 0.05...
                               ...                        
10204    [0.024017375, -0.0698705, 0.0030025216, 0.0155...
10205    [0.024017375, -0.0698705, 0.0030025216, 0.0155...
10206    [0.024017375, -0.0698705, 0.0030025216, 0.0155...
10207    [0.024017375, -0.0698705, 0.0030025216, 0.0155...
10208    [0.024017377, -0.06987051, 0.0030025218, 0.015...
Name: Embed_sentence, Length: 10209, dtype: object

In [89]:
# df.to_csv('amazon_review_embedded.csv')
# df = pd.read_csv('amazon_review_embedded.csv')

# df['Embed_sentence'] = df['Embed_sentence'].apply(np.array)
# df['Embed_sentence'] = df['Embed_sentence'].apply(eval)

# Find related reviews

In [23]:
topics_samples= [
    'Setup', 
    'Connectivity', 
    'Customer Support', 
    'Print Quality', 
    'Print Speed', 
    'Ink supply and Cartridge', 
    'Printer Hardware Robustness and sturdiness', 
#     'Control Panel', 
    'Ease of Use', 
#     'Firmware', 
    'Business Services and Subscription', 
     'Paper jam'
]

In [20]:
def embed_label(text,model):
    embeddings = model([text])
    return embeddings.numpy()[0]

text = 'Connectivity'
input_embedding_vector = embed_label(text,model)

df['similarity'] = df['Embed_sentence'].apply(lambda x: cosine_similarity(input_embedding_vector,x))

In [22]:
pd.set_option('display.max_colwidth', None)
df_new = df.sort_values(by = 'similarity',ascending = False)
columns = ['Brand','Review Model','Review rating','Review Content']
df_select = df_new[columns]
df_final = df_select[df_select['Brand'] == 'HP'].head(4).reset_index(drop = True)
df_final

,Brand,Review Model,Review rating,Review Content
0,HP,HP Smart Tank 6001,1,Connection problems
1,HP,HP ENVY Inspire 7955e,5,So easy to connect
2,HP,HP DeskJet 2755e,5,Connected to devices with no problem.
3,HP,HP Smart Tank 7301,2,wireless will not stay connected


In [25]:
text = 'Print Quality'
input_embedding_vector = embed_label(text,model)

df['similarity'] = df['Embed_sentence'].apply(lambda x: cosine_similarity(input_embedding_vector,x))

pd.set_option('display.max_colwidth', None)
df_new = df.sort_values(by = 'similarity',ascending = False)
columns = ['Brand','Review Model','Review rating','Review Content']
df_select = df_new[columns]
df_final = df_select[df_select['Brand'] == 'HP'].head(4).reset_index(drop = True)
df_final

,Brand,Review Model,Review rating,Review Content
0,HP,HP ENVY Inspire 7955e,5,I have been pleased with the print quality
1,HP,HP OfficeJet Pro 9025e,5,Print quality exactly. Best one I have had.
2,HP,HP ENVY 6055e,5,The print quality is good. I am very pleased.
3,HP,HP ENVY 6455e,5,Excellent print quality !!


In [34]:
text = 'Paper jam'
input_embedding_vector = embed_label(text,model)

df['similarity'] = df['Embed_sentence'].apply(lambda x: cosine_similarity(input_embedding_vector,x))

pd.set_option('display.max_colwidth', None)
df_new = df.sort_values(by = 'similarity',ascending = False)
columns = ['Brand','Review Model','Review rating','Review Content']
df_select = df_new[columns]
df_final = df_select[df_select['Brand'] == 'HP'].head(4).reset_index(drop = True)
df_final

,Brand,Review Model,Review rating,Review Content
0,HP,HP ENVY 6455e,4,Paper tends to jam
1,HP,HP OfficeJet Pro 8034e,5,the paper feed in front
2,HP,HP OfficeJet Pro 9015e,3,"what can I do, this printer continues to have paper jams and is not working"
3,HP,HP OfficeJet Pro 8025e,2,"If you print something? Paper jams. Reload the paper the only way it can go? Paper jam.WiFi blinks? Paper jam. You leave the printer alone too long? Paper jam. It’s incredible how hard it must be to make a printer that just prints when it’s told to, because no one can seem to do it. I’ve had this for 3 months and it’s a very high maintenance printer."


# Other (Draft)

In [19]:
# labels = [
#     'Setup', 
#     'Connectivity', 
#     'Customer Support', 
#     'Print Quality', 
#     'Print Speed', 
#     'Ink supply and Cartridge', 
#     'Printer Hardware Robustness and sturdiness', 
#     'Control Panel', 
#     'Ease of Use', 
#     'Firmware', 
#     'Business Services and Subscription', 
#      'Paper jam',
#      'Control Panel', 
#     'Other'
# ]

# df_labels = pd.DataFrame({'Labels': labels})
# df_labels

In [ ]:
# def docs2vecs(docs,dictionary):
#     vec1 = [dictionary.doc2bow(doc) for doc in docs]
#     tfid = gensim.models.TfidfModel(vec1)
#     vec2 = [tfidf[vec] for vec in vec1]
#     return vec2

# vectorizer = TfidfVectorizer(max_df = 0.5, min_df = 2, stop_words = 'english')
# X = vectorizer.fit_transform(dataset.data)

In [85]:
# model = Word2Vec(sentences = label_token, vector_size=100, window=5, min_count=1, sg=0)

In [112]:
# from gensim.models import Word2Vec
# import pandas as pd

# # List of labels
# labels = [
#     'Setup', 
#     'Connectivity', 
#     'Customer Support', 
#     'Print Quality', 
#     'Print Speed', 
#     'Ink supply and Cartridge', 
#     'Printer Hardware Robustness and sturdiness', 
#     'Control Panel', 
#     'Ease of Use', 
#     'Firmware', 
#     'Business Services and Subscription', 
#     'Paper jam',
#     'Control Panel', 
#     'Other'
# ]

# # Create a DataFrame with labels
# df_labels = pd.DataFrame({'Labels': labels})

# # Split the labels into words (tokens)
# label_tokens = [label.split() for label in labels]

# # Train a Word2Vec model on label tokens
# model = Word2Vec(sentences=label_tokens, vector_size=100, window=5, min_count=1, sg=0)

# # Function to get the embedding of a label
# def label_embedding(label, model):
#     return model.wv[label]  # Use indexing to retrieve the embedding

# # Apply the label_embedding function to embed labels
# df_labels['Label Embeddings'] = df_labels['Labels'].apply(lambda x: label_embedding(x, model))

In [19]:
# def search_reviews(df, product_description, pprint=True):
#     product_embedding = get_embedding(
#         product_description,
#         engine=embedding_model
#     )
#     df[f"similarity_{product_description}"] = df.embedding.apply(lambda x: cosine_similarity(x, product_embedding))

#     results = df.sort_values(f"similarity_{product_description}", ascending=False)
                       
#     return results



# summary_df = []

# for label in labels:
#     label_results = search_reviews(df_filter, label, pprint=True)
    
#     # Apply a similarity threshold to filter rows
#     threshold = 0.85
#     filtered_results = label_results[label_results[f"similarity_{label}"] > threshold]
    
#     # Add the label column to the filtered results
#     filtered_results['topic'] = label
    
#     # Append the filtered results to the summary_df
#     summary_df.append(filtered_results)
    
#     time.sleep(5)

# # Concatenate the filtered results into the final summary DataFrame
# summary_df = pd.concat(summary_df)


In [30]:
# columns_to_drop = ['embedding', 'embedding_splitted']

# # Use the drop method to remove the specified columns
# df_phrase_filtered = df_phrase.drop(columns=columns_to_drop)

In [38]:
df_filter = df_filter.drop_duplicates(subset = ['Review Model', 'key phrase splitted'])

In [46]:
def find_most_similar_word(review_embedding, word_vectors, word_list):
    similarities = [cosine_similarity(review_embedding, word_vector) for word_vector in word_vectors]
    most_similar_index = similarities.index(max(similarities))
    return word_list[most_similar_index]  # Return the actual word

# Assuming you have a DataFrame 'df' with 'embedding' column containing review embeddings
df_phrase['Topic'] = df_phrase['embedding_splitted'].apply(lambda x: find_most_similar_word(x, label_embeddings, labels))


C:\Users\YeTi985\AppData\Local\Temp\ipykernel_17244\3029389664.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_phrase['Topic'] = df_phrase['embedding_splitted'].apply(lambda x: find_most_similar_word(x, label_embeddings, labels))


In [47]:
df_phrase['Topic2'] = df_phrase['embedding'].apply(lambda x: find_most_similar_word(x, label_embeddings, labels))

C:\Users\YeTi985\AppData\Local\Temp\ipykernel_17244\3720456180.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_phrase['Topic2'] = df_phrase['embedding'].apply(lambda x: find_most_similar_word(x, label_embeddings, labels))


In [48]:
df_phrase 

,Unnamed: 0,Review Model,Retailer,Review date,Review rating,Review title,Review content,Review concat text,Tokens,key phrase,embedding,key phrase splitted,embedding_splitted,Topic,Topic2
6388,6388,HP OfficeJet Pro 9025e,Amazon,2021-05-07,3,Easy install and easy printing. Too proprietary.,Easy set up. Easy scanning. Didn’t join the i...,Retailer: Amazon - Review model: HP OfficeJet ...,198,"[Easy setup, easy scanning, Proprietary restri...","[0.00624576723203063, 0.003082588315010071, 8....",Easy setup,"[0.0028877160511910915, 0.015896936878561974, ...",Setup,Ink supply and Cartridge
6389,6389,HP OfficeJet Pro 9025e,Amazon,2021-05-07,3,Easy install and easy printing. Too proprietary.,Easy set up. Easy scanning. Didn’t join the i...,Retailer: Amazon - Review model: HP OfficeJet ...,198,"[Easy setup, easy scanning, Proprietary restri...","[0.00624576723203063, 0.003082588315010071, 8....",easy scanning,"[-0.04566308483481407, 0.025933345779776573, 0...",Ease of Use,Ink supply and Cartridge
6390,6390,HP OfficeJet Pro 9025e,Amazon,2021-05-07,3,Easy install and easy printing. Too proprietary.,Easy set up. Easy scanning. Didn’t join the i...,Retailer: Amazon - Review model: HP OfficeJet ...,198,"[Easy setup, easy scanning, Proprietary restri...","[0.00624576723203063, 0.003082588315010071, 8....",Proprietary restrictions,"[0.0037913022097200155, -0.012775414623320103,...",Firmware,Ink supply and Cartridge
6391,6391,HP OfficeJet Pro 9025e,Amazon,2021-05-07,3,Easy install and easy printing. Too proprietary.,Easy set up. Easy scanning. Didn’t join the i...,Retailer: Amazon - Review model: HP OfficeJet ...,198,"[Easy setup, easy scanning, Proprietary restri...","[0.00624576723203063, 0.003082588315010071, 8....",Inflexibility with ink brands,"[-0.0056190043687820435, 0.005538931582123041,...",Ink supply and Cartridge,Ink supply and Cartridge
6401,6401,HP OfficeJet Pro 9025e,Amazon,2023-03-03,5,Great Printer,"After years of my old printer, purchased this ...",Retailer: Amazon - Review model: HP OfficeJet ...,84,"[Fast printing, liked two tray feature, easy w...","[-0.03244659677147865, 0.009338131174445152, -...",Fast printing,"[-0.0462610200047493, 0.024372506886720657, -0...",Print Speed,Print Speed
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10711,10711,HP OfficeJet Pro 9025e,Amazon,2021-05-08,5,Great for working from home,"This really is a great printer, and it's so mu...",Retailer: Amazon - Review model: HP OfficeJet ...,268,"[Excellent for home office use, works seamless...","[-0.014914722181856632, 0.0015333282062783837,...",positive response to free ink plan,"[-0.013421832583844662, -0.01247564610093832, ...",Ink supply and Cartridge,Printer Hardware Robustness and sturdiness
10712,10712,HP OfficeJet Pro 9025e,Amazon,2021-05-02,5,"Really Nice, Great Setup",So the setup process with printers is normally...,Retailer: Amazon - Review model: HP OfficeJet ...,132,"[Easy setup process, App-based setup is conven...","[-0.016588890925049782, 0.017046066001057625, ...",Easy setup process,"[0.006355913821607828, 0.021701103076338768, 0...",Setup,Ease of Use
10713,10713,HP OfficeJet Pro 9025e,Amazon,2021-05-02,5,"Really Nice, Great Setup",So the setup process with printers is normally...,Retailer: Amazon - Review model: HP OfficeJet ...,132,"[Easy setup process, App-based setup is conven...","[-0.016588890925049782, 0.017046066001057625, ...",App-based setup is convenient,"[-0.006079057231545448, 0.010124939493834972, ...",Ease of Use,Ease of Use
10714,10714,HP OfficeJet Pro 9025e,Amazon,2021-05-02,5,"Really Nice, Great Setup",So the setup process with printers is normally...,Retailer: Amazon - Review model: HP OfficeJet ...,132,"[Easy setup process, App-based setup is conven...","[-0.016588890925049782, 0.017046066001057625, ...",Supports multiple connections,"[-0.038474854081869125, 0.005594192072749138, ...",Connectivity,Ease of Use
